In [1]:
import sagemaker
from sagemaker import Session, image_uris
from sagemaker.model import Model
import pandas as pd

# SageMaker session & execution role
session = sagemaker.Session()
role = sagemaker.get_execution_role()

print("Default Bucket:", session.default_bucket())
print("Execution Role:", role)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Default Bucket: sagemaker-ap-southeast-1-190403256083
Execution Role: arn:aws:iam::190403256083:role/CrowdRiskRole


In [8]:
import boto3

sm_client = boto3.client("sagemaker")

response = sm_client.list_endpoint_configs()
for ec in response['EndpointConfigs']:
    print(ec['EndpointConfigName'])


crowd-risk-option1-v2


In [9]:
import sagemaker
from sagemaker import Session, image_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor

session = Session()
role = sagemaker.get_execution_role()
region = session.boto_region_name

container = image_uris.retrieve("xgboost", region, version="1.5-1")
model_s3_path = "s3://sagemaker-ap-southeast-1-190403256083/final-xgb-model-option1/sagemaker-xgboost-2025-09-20-18-01-11-206/output/model.tar.gz"

endpoint_name = "crowd-risk-option1-v2"

sm_client = boto3.client("sagemaker", region_name=region)

try:
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"✅ Deleted existing endpoint: {endpoint_name}")
except sm_client.exceptions.ClientError:
    print(f"⚠️ No existing endpoint to delete: {endpoint_name}")

try:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"✅ Deleted existing endpoint config: {endpoint_name}")
except sm_client.exceptions.ClientError:
    print(f"⚠️ No existing endpoint config to delete: {endpoint_name}")

xgb_model = Model(
    image_uri=container,
    model_data=model_s3_path,
    role=role,
    sagemaker_session=session
)

xgb_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)

# Create predictor object explicitly
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=session)
print("✅ Deployment completed. Endpoint name:", predictor.endpoint_name)


✅ Deleted existing endpoint: crowd-risk-option1-v2
✅ Deleted existing endpoint config: crowd-risk-option1-v2
------!✅ Deployment completed. Endpoint name: crowd-risk-option1-v2
